In [12]:
# %pip install -qU langchain langchain-core langchain-community langchain-ollama langchain-groq python-dotenv pypdf chroma chromadb

Note: you may need to restart the kernel to use updated packages.


# Libraries

In [1]:
import os, getpass, textwrap, time, uuid
from dotenv import load_dotenv # type: ignore



from langchain_community.callbacks import get_openai_callback # type: ignore
from langchain_core.runnables import RunnablePassthrough # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore

from langchain_groq import ChatGroq # type: ignore

In [2]:
# Load environment variables from .env file
load_dotenv()

True

# Helper Function

In [3]:
def pprint_docs(docs):
    print(f"\n{'-' * 70}\n".join([f"Document {i+1}:\n\n" + "\n".join(textwrap.wrap(d.page_content)) for i, d in enumerate(docs)]))

def pprint_result(result):
    print("Answer: " + "\n".join(textwrap.wrap(result)))

# Load PDF File

In [1]:
from langchain_community.document_loaders import PyPDFLoader # type: ignore

# load the PDF  
file_path = "data/5. Pedoman Menteri PANRB NO 3 Tahun 2024 Pedoman Tata Cara Pemantauan dan Evaluasi SPBE.pdf"
loader = PyPDFLoader(file_path)

documents = loader.load()

In [2]:
documents[1]

Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-07-15T14:04:57+07:00', 'author': 'Muthia N R', 'moddate': '2024-07-15T14:04:57+07:00', 'source': 'data/5. Pedoman Menteri PANRB NO 3 Tahun 2024 Pedoman Tata Cara Pemantauan dan Evaluasi SPBE.pdf', 'total_pages': 236, 'page': 1, 'page_label': '2'}, page_content='-2- \njdih.menpan.go.id \nUpaya untuk mendorong perkembangan keterpaduan penerapan SPBE \npada Instansi Pusat dan Pemerintah Daerah dilakukan melalui penerapan \nArsitektur SPBE. Arsitektur SPBE Nasional telah ditetapkan melalui Peraturan \nPresiden Nomor 132 Tahun 2022 tentang Arsitektu r SPBE Nasional, sehingga \npembangunan layanan digital pada Instansi Pusat dan Pemerintah Daerah \nharus selaras dengan Arsitektur SPBE Nasional. Selain itu, pada tahun 2023 \ntelah ditetapkan Peraturan Presiden Nomor 82 Tahun 2023 tentang Percepatan \nTransformasi Digital dan Keterpaduan Layanan Digital Nasional, yang menjadi \nsalah

# Chunking the SPBE with Parent Document Retriever

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore

# Create a splitter for the parent documents
text_parent = RecursiveCharacterTextSplitter(chunk_size = 2000)

# Create a splitter for the child documents
# Note: child documents should be smaller than parent documents
text_child = RecursiveCharacterTextSplitter(chunk_size = 400)

# Load Embedding Model

In [4]:
from langchain_ollama import OllamaEmbeddings # type: ignore

# load embedding model
embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

# Load Chroma as Vectorstore

In [5]:
# Initialize a vector store to storing the chunks
from langchain.vectorstores import Chroma # type: ignore

parent_vstore = Chroma(
    collection_name="spbe_parent_chunk",
    embedding_function=embeddings,
    persist_directory ="./chroma_spbe_parent_db"
)

# initialize in-memory storage for the parent chunks
from langchain.storage import InMemoryStore # type: ignore
store_parent = InMemoryStore()

C:\Users\regim\AppData\Local\Temp\ipykernel_21736\1118463972.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  parent_vstore = Chroma(


# Parent Document Retriever

In [6]:
# Create a parent document retriever
from langchain.retrievers import ParentDocumentRetriever # type: ignore

parent_retriever = ParentDocumentRetriever(
    vectorstore=parent_vstore,
    docstore=store_parent, 
    child_splitter=text_child,
    parent_splitter=text_parent,
    search_type="mmr",
    search_kwargs={"k": 2, "score_threshold": 0.8}
)

# add documents to vectorstore
parent_retriever.add_documents(documents)

In [15]:
# Cek jumlah dokumen yang tersimpan
print(f"Jumlah dokumen dalam ChromaDB: {parent_vstore._collection.count()}")

Jumlah dokumen dalam ChromaDB: 4004


# 4. Load LLM

In [16]:
# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY not found in .env file")


model = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
)

# 5. Prompt Template

In [17]:
from langchain_core.prompts import ChatPromptTemplate # type: ignore
# Log: Initializing prompt template
print("Initializing prompt template...")

prompt_template_naive = """
Anda adalah asisten asesor SPBE yang detail, khususnya dalam memberikan informasi dan membantu auditing Sistem Pemerintahan Berbasis Elektronik (SPBE) Indonesia.

## 🎯 Tugas Anda:
1️⃣ Menjawab pertanyaan umum tentang SPBE secara akurat.  
2️⃣ Menunjukkan domain, aspek, dan indikator juga level terkait dengan pertanyaan pengguna berdasarkan pedoman SPBE.

## 📢 Tanggapan Anda harus mengikuti aturan berikut:
- Jika pengguna bertanya tentang suatu domain, tampilkan nomor doman, aspek, dan indikator yang relevan, lalu tampilkan isi level tersebut.  
- Jika pengguna hanya ingin ringkasan, berikan jawaban singkat. Jika mereka ingin detail, berikan isi pasal lengkap.   
- Jika pertanyaan tidak tercakup dalam Pedoman SPBE, berikan respons berikut:
  - *"Maaf, pertanyaan tersebut tidak tercakup dalam Pedoman SPBE KemenpanRB. Saya hanya dapat memberikan jawaban berdasarkan pedoman tersebut."
---

## 🔍 Konteks yang diberikan:
{context}

## ❓ Pertanyaan pengguna:
{question}

## ✅ Jawaban Anda:
"""

# Log: Creating ChatPromptTemplate from template
print("Creating ChatPromptTemplate from template...")

prompt = ChatPromptTemplate.from_template(prompt_template_naive)

# Log: ChatPromptTemplate initialized successfully
print("ChatPromptTemplate initialized successfully.")

Initializing prompt template...
Creating ChatPromptTemplate from template...
ChatPromptTemplate initialized successfully.


In [18]:
questions = [
    'jika "Rencana dan Anggaran SPBE instansi pusat/pemerintah daerah telah terpadu dan dapat dikendalinkan oleh unit kerja/perangkat daerah yang menjalankan fungsi perencanaan dan penganggaran dan telah direviu serta dievaluasi secara periodik." maka dapat diberikan level...'
]

# Function Invoke

In [19]:
def do_retrieval(chain):
    for i in range(len(questions)):
        print("-" * 40)
        print(f"Pertanyaan: {questions[i]}\n")
        with get_openai_callback() as cb:
            pprint_result(chain.invoke(questions[i]))
            print(f'\nTotal Tokens: {cb.total_tokens}\n')

In [20]:
parent_chain = (
    {"context": parent_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [21]:
do_retrieval(parent_chain)

----------------------------------------
Pertanyaan: jika "Rencana dan Anggaran SPBE instansi pusat/pemerintah daerah telah terpadu dan dapat dikendalinkan oleh unit kerja/perangkat daerah yang menjalankan fungsi perencanaan dan penganggaran dan telah direviu serta dievaluasi secara periodik." maka dapat diberikan level...

Answer: Berdasarkan Pedoman SPBE, pernyataan tersebut terkait dengan Domain 1:
Perencanaan dan Penganggaran, Aspek 1.2: Perencanaan dan Penganggaran
SPBE, dan Indikator 1.2.2: Rencana dan Anggaran SPBE terpadu dan
terkendali.  Level yang dapat diberikan untuk pernyataan tersebut
adalah Level 4, karena telah memenuhi kriteria sebagai berikut: -
Rencana dan Anggaran SPBE telah terpadu dan dapat dikendalikan oleh
unit kerja/perangkat daerah yang menjalankan fungsi perencanaan dan
penganggaran. - Rencana dan Anggaran SPBE telah direviu serta
dievaluasi secara periodik.  Isi Level 4 adalah: "Rencana dan Anggaran
SPBE telah terpadu dan dapat dikendalikan oleh unit kerja/p

In [22]:
pprint_docs(parent_retriever.invoke(questions[0]))